# LangChain Model

模型是Agent的执行引擎，不断推动Agent的决策、执行、分析结果最终完成任务。

## 创建模型

模型常见有两种使用方式。

1. 创建Agent时指定模型。
2. 独立使用模型完成任务。

下面是创建模型的方法

In [ ]:
from dotenv import load_dotenv

load_dotenv()

from langchain.chat_models import init_chat_model

llm = init_chat_model("deepseek:deepseek-chat")

response = llm.invoke("Why do parrots talk?")

print(response)

## 模型支持的参数

1. model:LLM厂商和特性的模型名称
2. api_key:用于身份验证的秘钥
3. temperature:控制模型输出的随机性。数字越高，响应越有创意；数字越低，响应越确定。
4. timeout:在取消请求之前，等待模型响应的最长时间。
5. max_token:限制模型输出的最大Token数。注意，这个限制很有可能导致模型输出的内容被截断。
6. max_retry:如果请求因网络超时或速率限制等问题而失败，系统将尝试重新发送请求的最大次数。

## 调用方式

### invoke

直接方法是使用invoke()，带一个消息或一个消息列表。

In [ ]:
from dotenv import load_dotenv
load_dotenv()
from langchain_deepseek import ChatDeepSeek
llm = ChatDeepSeek(model="deepseek-chat")
response = llm.invoke("which is greeter beetween 3 and 4")
print(response.content)

### 流式输出

大多数模型可以在生成输出内容时进行流式传输。通过逐步显示输出，流式传输显著改善了用户体验，特别是对于较长的响应。

In [ ]:
from dotenv import load_dotenv
load_dotenv()
from langchain_deepseek import ChatDeepSeek
llm = ChatDeepSeek(model="deepseek-chat")
for chunk in llm.stream("which is greeter beetween 3 and 4"):
    print(chunk.text, end="", flush=True)

与invoke()不同，invoke()在模型完成生成完整响应后返回单个AIMessage，而stream()返回多个AIMessageChunk对象，每个对象包含输出文本的一部分。重要的是，流中的每个块都可以通过求和聚合为完整的消息。

### 批量处理

批量处理对模型的独立请求集合可以显著提高性能并降低成本，因为处理可以并行完成。

In [ ]:
from dotenv import load_dotenv
load_dotenv()
from langchain_deepseek import ChatDeepSeek
llm = ChatDeepSeek(model="deepseek-chat")
responses = llm.batch([
    "为什么鹦鹉有彩色的羽毛？",
    "飞机是如何飞行的？",
    "什么是量子计算？"
])

for response in responses:
    print(response.content)

默认情况下，batch() 将仅返回整个批次的最终输出。如果您希望在每个单独的输入生成完成后接收其输出，您可以使用 batch_as_completed() 进行流式传输。

## 工具调用

模型可以请求调用工具来完成任务。可以是mcp tool也可以是function call的方式。

In [ ]:
from dotenv import load_dotenv
load_dotenv()
from langchain.tools import tool

@tool
def get_weather(location: str) -> str:
    """Get weather at a location"""
    return f"It is sunny at {location}"

from langchain_deepseek import ChatDeepSeek
llm = ChatDeepSeek(
    model="deepseek-chat"
)
model_with_tools = llm.bind_tools([get_weather])
response = model_with_tools.invoke("What's the weather like in Boston?")
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")
print(response.content)

## 结构化输出

有些时候需要模型输出特定结构规范大的数据，方便后续处理。LangChain提供多种模式类型和结构化输出的方式。

### Pydantic

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from pydantic import BaseModel, Field

class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

from langchain_deepseek import ChatDeepSeek
llm = ChatDeepSeek(
    model="deepseek-chat"
)
model_with_structure = llm.with_structured_output(Movie)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)

## 高级主题

### 多模态

某些模型可以处理非文本类型的数据，包含图像、影音、视频等信息，可以通过提供内容块的方式提供给模型。

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain_deepseek import ChatDeepSeek
model = ChatDeepSeek(
    model=""
)
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Describe the content of this image."
            },
            {
                "type": "image",
                "base64": "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAADEAAAAyCAYAAAD1CDOyAAAQAElEQVR4AVxae7BfVXX+vn1+931z85JEAZUBC1TSiEphcCIIKgktFAawqBSV6R8F+rIVhIK2UKtWqaXKs7XQMkh5KjMM7SDY4SU4BTJVgVAIgRASuDcJScjz3vv7nbP7feucc+/VnbPOWnuttdf61tr7nPMjJL02MZ5N68ffzBs2T+SXN76e123amF9+fUN+peFrN7wW8jrZrG+59ZZNIb/+el772mt5nfjLGzbkX6dXNm7ML7V2xVqrHM7X8peUx7k9N1m23fFMXtvGXKv4a5XntfHxnEiCrGl6agp9nQ6GBwcxf2wM80ZHMTZvXtDw0FDMrTNZb5/RkZHQe26yzvaWW7be3L7zHU+xR4dHIu7YqOIrj+1eM2Z5pM47OjwcsedJ57ULtM5ykPL2dzroTk8jlWWJnDN63S5GtGhwYAAppdAhI0ZiikI9IRk2rzFZF1y+VVWFX8xtEKWiCH+JM1elnAoDGXRVosYkHGVZxSRXCiiJoO6zl7UkYYxDavaIipGc0FUBfX19YWgBc87iKlcgBV6BDZAk2mF/ywpkJkBOE2KsMWDPyNk1nssIk3CDifAfUHfLImjYlpVbokzKL0WREko1K0u2vkgJyQLJOEYGZMB2yNqGoMY5OkN7/yrZ3xpzsk5E1txxbDNZbncqCnZckQtQ5QgblEA6+4feQkMk4XU97WJHu4tmOG4U4W4lORk0qUCNgxlJuDjbHISsAZIEWdtIwsMBzU2W7W+ZZO2rrpEMwNZLCRdgWWqJlcUZatc7lpXmLsC8nZNEig5IYJPAHQ/QOjrQ8AIXIBizyVu9uuYdsI/XkPISxVzxHFuuwkoBrI+ZdTPglKOV7SdHBEFDsXXXNIk4k9uxrTeRjLjaAHoOg7cDE2FgqNUwONusD0ffhIckSCLWiMca2QxSDOZk7eO5yb7mtpmbWrnBPLMzCq7Yfujr3SFnYzmOizcXFCQHIgkroSFX3aEYhEeAk+gFns/lhP5ore42geo+yVoWt6+pBRoG3cjaR6KA1ritouLBQlNRMM/t2JDjWXRMY6Ym4lGHxPoyaNKmeu67nMwUnzVpt2qFAOhIOKDnVVUC1GX7HA4Nx0hFAimD5gRRJM0BJOmoedZbB4pnubYAVKszlUekfYGcZ0imkJMrMxk8mkEFbESQhEEmJbSfbeZxzNQqJoZrcsS3d4Av/R/w8IOorv8O8sUXAueeieqsleh+ahXK885G/upforz1+6h+9hiqV9ch7d0DR3BMByI9Q7xGUyM7P8PLHiLl1T1wZRUdRVihN6q2NoOkxJqHvrlFINlcLEmYt4nT1CTKB+8Hrr0K1VVXonf91cBDDwAvr0Xes1O+GfqaAtu3Ij//S+BHd6K65tsov3kFyhuvRvX0z5D16mxSBY42Nkk1n9JV8IhipDP4mQZWzRcyHFojqP2zRqypWpVJS5AKqOo9N1GdzDdcjfyDm1Cufhr5ra1gr4uejH6rGUzSByuOhWI5uaKCe/cBE5vQe/xhFX0VeP+P/PMhkpJ1HpKom+dfEGGKqLLCJ8Ma8+RzmnRUTFaCTpEx02k1kSSYGAFaH1p4cQ3Kr12G7uM/AXbv0vmVsy61DYUBBwFdBXVRCuALPflkS7L7GOZtO9C95V/RvfNWZB1JAy9Sgj9sZJ1XzBkFzisz7EPFzWpQygpkRe0BkDLNIReJOaPupBQvvoD8L99DJY6KCq5LAbOoJ7P9Ku1YVvxCD2ySXEl2MZSP7fazzkWj7KK653ZU378OactEgPSHTaEcOFwsk8ksKCuOhdCQRNUoCMlKam4HNyy4blkgqB3xEar+/UZ09WDSbyTpbbOvuQFOy1/ZlTwH9eRjUi2AbvZLymMuB13y602j++hP0L31ZrA7PbsT2jmfFGPKxqlYgqw1AJkgWwJJtEOhYu6iCBXkRY0x3kh79qD33aswteZZUCB8NBwtK3BNACV3RLpgm4srgDhiHa+RLFxhquRk8kQvaK2t0HvyYVQ/fURyRiV/gQyeHUhr0eA1y8KXyl4ZDnZ0B7OC1n6zBbS6rrZ8zZqHse2F1TCo7AQiLzEZTPgqel64EIMf/RjG/uRLWHTF32PR3/wdFn75PCy44EgsPm8MY+d0sOBzCYs+V4gsF1h8bpKuwKJPlxjqfwCp95ahBGbqBGTtoBUuzLylVHQMB1GIlSTVlAzqjxe5+2St2zG9C7dPPov/PqgfVdOVrEUGntURLQQHBjH88VVY8q1rsODyb2Dfys9g/eGfwKZlq4BPXoDhs6/DyGl/jZEVh6NvWSEiBj8ADIn6lzH40AeI/v1eRv/0C6Bea25QxFYhZNJHslDW2SsZqPEQDC3FTZ6QNfiYC+2WyW1YP70Fd31wDM8vHYA7b8rKUoqgAkbPPhejf3YJXp1/OP7xEeAv7pjEJXftxUV37MWf/0cXNz/RwRvpY+B7LwXnfRCdQockl0pXCVwprPoeuCHdnejsfkRHalI2XVmknfAuZO0+hNO4TTpFCSThQdbcoOxMzs7liHU71mPrvrexp4+468MLsWMwISuw3zgKgr4VJ2DoU3+A+9YOBfAfPjONl8ZLvLGjwqbtFX6xoYebHp3E5fdM4sVd2olDLgb6l6LUg5VVSKWXBKGCJHeKCp0dT+oB325ocHzqtRuIGlyeQ0MvCFUuwZfBmwyYrHeh1fudvX73G+q+kqj8F5f04wXthvoYDyzGFmD+aWfi1X0juPWJSWzcVurtkuXvFmY4bqXdmtbyF97o4d7VXeztvA/c/wsIgLLF0YkuZ1AHtoPd6Nv5WKzVDVk2A3cskjGHeLLCBA1SBgVrd4GkfGryO/vZbS/CvqZ9HeKxQ0awXbtRajeGlh+JzmFH4M6npqPzWfFMYioZs6T4PX1XHn+pi+c3leDY0egMLZFbBQq4PbN3Q3IWdfY8LVtzCY9zu8nmEV+6RDI8rCQJO5hcCDTMbTPfsGMcWYArkZ+B1QcO4cmDRjCdiMEPHY1t3X48u1E70NjFYifsW8vQHPHjbmJnhadf6aFXqIDBAwEdoZaoZ6LXrQAdr2LfGpDCaFIDIDIe89gV7U4Szl+57FDZoPNnA8kI4qqnyym4CDSB9vYT9x45H7v1jKRFi7FlD7BrsgqgXpvVf4MXky4HOb5tlQzrt5To5iGwGFFIgVbnyQq9XqUHvgI1z9PbUQmPHAAyqI1hHUkkUgbUw852ILXcC2t13OUIB4nf9qooC5l9J4Y6uG/5GKbKafQV4QKPLB/htChPzFAljUyCBwz2QyGzGl4ifsR6kXahU1TCZ8pgUcIj+9YQKcxzmpwMvLEpIIPauUG2RBL7DSxAVnGVtjkLYW2r8NBho/jfqTex/1iFAxbqjaUAfohtNzfNlX28Cp2BIw7ooD9v1xtoHJ3UU6Ulso6V31bQc5GVh33vUrT6cgwBjCMvx1BapwYzJr5ZYbLcEkmto9ZkvHNY51e5gHpu30rv+Z2dAve+/UukootjDu5D0k/TmY6rhaoXpiy5pXcvSlh+YAecHkdv9yuA9qb0fxZoJyqBh7iPVnfw/SCpmKoa9ajUSOf2zDwsRVGEo5VkDdCyyU4kBSLjkPnvQVbwmiBZHkJVdSqsrjbjmc1rsHJZB4e9s6NzCq2RvbmEf+ZIDeoZ+uQR/Th4iY7M5rtR5Prct+D6FM/Hq8oZvbHjo4AsuQkVc5LBoRFFtIs1FzCns1RTas5ekRKOfMfhGCz6ah9/oJqOOMHOqd245plbsb1ch6+cOoQzjxrA4hEKeIaPj30GOmrEkoQvrBjE6R+usHfibpQT9yCX2l4doyLp2dBDracERdFDNXAguiNH6nkpo8ktFhcnBYybJFRE1pzw0FwygqDheVbn7ewtPnDe/lg4shBZv7MrvUWyfLLOSVUKqAp6fstafOepm9E/vAkXrRzGN84axfknDGHVb/XjtA8N4LJTRvHt35+H847rYMPWH2PHq98De96FrJxVUKcjwCqo7Gb05h2P3FmsLFDjqporTxYFYt1SIhJJMBFzR71zOVSWKXNKBZYML8byxYeDHT28ipl1hrMdSifJ6niFpzb9Ahf811fxgzV34IAlEzh3BfD1M0Zx+amDOOGIfSg76/BvP78N//PcP2G/3mta2ENungGD0wRZjcudeZgePRZgP1JSvhqOCtUS7S80shro9NoJxCvOjtIrgL2znIVcChdgx0rVD3cGcOJ7jsXi/vkKk2uSMSdVJN8sH/ttfHsc333qFvzh/X+FL/74Slz6yNdw6UNfxx//5xX4/H2X4O6f34KP9m9AX9WDDxy1q0KhCI5T6fioIfOOQTmyXDkYx0bGGV6oodkWQTRPqSgCuJPbUQqQlM6zllxULf/20uX49G+eggKd8FENDa9mOYGeurtx55t4ctNqPLDuMTy44XGs3vwcdk7twnGj23HE8G5AHdcDofOvrdTbiSKY1KTuks8jFwsiKRMRxJjWxThxPYX+o6jXiID1LsZ8RikhFXWhAH3+cOZhK3H2YSejL9cPeWx/bK0K0fORQ87Q3xBol9UrzSsRRIekSVyw3xvo03dAGZXTayr421BqbRdj2HfAZZge/iDIBA/HS2wqkIJJsucmgQ0vktAV5MAphVrumglL1jEBiHYUSPjs+38Pxx90tFTqn5IbZO7KvycS2EpryjhmWdgdJGN+6uJL+2/AkmJK0UrtQE+8PkLwrvSPYOrdKmD+7wKpowMDOI5Bm0Mg480k4DApu3WJFAglzY2SJOBLBI22IKutzxZEY/3zcOGR52DlQcehP8lR/lUDOtOggey4wujY7AEnz9+GDw3sRP0MZCgz3H2/glHMx+S7LkV34WlAGtDirKASU5IfkcQhjAJsFgQP61xhSvQ0iEw6AsqsmW2VzrbEepHAW7ZPoSO2dOQduOzY83HxsRdi2eJD4xtSpaxjLgC9DPaobmYBApYOTmLl/K0Y1s+LrK5n75RNeoV2F52KPYfejKl3fBZIg4DzmKDRcGPRDLZReNuiyIREEh4pFaAUdrbKPKVkU11ASMKj7loUVFD+ffr4nfK+j+PKFV/El485Hye+9yMY7AzAxZTUbyG97w36qOGd+MDQLvSlUkckoxw8GNNLL8De3/hnTL7nm+iNHgWyUNHOoSaqw21ir6dyQcAINUYYnB+a2yacRQT1xMCl19ose10AwLjMZIDtaEYEUrKEhAPGlmLVwcfjKyv+FDf9zrdwiXbomAOWY2hoID6Opy3YjmLwUEwtPgP7DvoH7Drsh5jc/yKBPwZZR4lgExUgJbeEerS5jJNJduW1hXR2SYWORqMzTmnqyzrHMq819d2BWkfbXLwtRSowUgzg0MUH4azDT8a1J/0tHj3nTjzzR/dh2XHPYc+yh7HnvdeqkM8g9y1FhX5Qf7w2K5Dj+Jh53sZnTHSTHQYjIigFkCRnHc3U/i/g0M7cGJK2SZzNEm1j1jRmlEBJJohDZwAIfwf21ElTvZsGR8nZOrL2k0wS7g6a9wAAAnxJREFUMSS3oGPuW6OLtUygCDpGcd7CrpQNHuVNVoFsAsZs9ubE4aug4SFua3RBD6flIK13Qsvhb0FEElGAZRVCUm+kMvKRbkzjLVlgZvXyd24x1ZdBCyblb3fLGyAjogI7m8g5QbXAOrE6gGxoyHq/r2lQDTmY9eHjRSYlNDOANrF9SGuA0Mkn28lcFKKaY48g+0ofPpLncjfS+ZKDzu3AXNk2UqFEJOVP50DcFdidNxD72eC1nluG/M3n2ubO5/qRimhSzNbfvm4SrNMk/CXLUzOgSNF/dSJDxaj7UntrApQdFZCktPaJ2uEgJrLW2xiS5l7nuTkZWmxYvx6333YbbrzuOtwguv7aa3HDNdfg7rvuwpvj43afJeU0+DqTcsrSyhRYN0cquDHWk4RzwUNylEMSpbbQOlMEVGCScAAvMIeGbWL1JTvkp07ACZzQ3LqfPv443t6xI55Dr4nS5L/trbfwxGOPhX8bE83wnKRC1GS115q3RLIVEfmc3xqS2LN3bzxwcwM5gMkg5xapLAhSAHemleEhneekkolIakroBt3hAlvu5ngODefxTsfcMUTWeR5cPqRWtnrJ1k/5X9mQBEkU+lZsV+d27d6Nqamp2C6SihEwQdTDC031DDP66Apmx4rjjsPY2FiAVpCaK94C/ZX/R2SD5Llr3Kh2dZ1RMwG2n6S4nDds0vsf1ezdtw87d+1CPNj28L+yGR7R3+Z1u9ipQjZv2YItW7fCfLO5aEK6ic2bQxe80dnPc3P7+swPKdYnVq3C6WedhTPOPjvo9DPPxIknnYS+/n7Yf3xiAhFTcb1u3LEV03Favf2C5GPe4jHGXlliaHgY/w8AAP//aBaDIgAAAAZJREFUAwDXgOV/D2eaygAAAABJRU5ErkJggg==",
                "mime_type": "image/jpeg",
            }
        ]
    }
]
for chunk in model.stream(messages):
    print(chunk.text, end="", flush=True)

### 推理

有些模型支持多步推理的方式完成任务，将复杂的问题划分为更小、更便于管理的步骤。

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain_deepseek import ChatDeepSeek
model = ChatDeepSeek(
    model="deepseek-reasoner"
)
for chunk in model.stream("解释下为什么3、4、5可以构成直角三角形？"):
    reasonings = [r["reasoning"] for r in chunk.content_blocks if r["type"] == "reasoning"]
    reasoning_str = "".join(reasonings)
    print(reasoning_str if reasoning_str else chunk.text, end="")

### Token用量

许多模型提供商在调用的时候会返回Token的使用信息，这个信息会存在于AIMessage中。LangChain对于支持的模型，可以通过UsageMetadataCallbackHandler来获得。

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain_deepseek import ChatDeepSeek
model = ChatDeepSeek(
    model="deepseek-chat"
)

from langchain_core.callbacks import UsageMetadataCallbackHandler
callback = UsageMetadataCallbackHandler()
response = model.invoke("Hello", config={
    "callbacks": [callback]
})
print(response)
print(callback.usage_metadata)

### 调用配置

调用模型时，你可以使用 RunnableConfig 字典通过config参数传递额外的配置，这提供了对执行行为、回调、元数据跟踪的运行时配置。

常见的配置包括

```python
model.invoke(
    "Tell me a joke",
    config={
        "run_name": "joke_generation",      # Custom name for this run
        "tags": ["humor", "demo"],          # Tags for categorization
        "metadata": {"user_id": "123"},     # Custom metadata
        "callbacks": [my_callback_handler], # Callback handlers
    }
)
```